### Reorder not allow

In [ ]:
import copy
import random
# reorder fail 21:53
C = [0 for i in range(32 * 4)]
C_idx = [[] for i in range(32 * 4)]
A = [random.randrange(-5, 5) for i in range(32 * 4)]
B = [random.randrange(-5, 5) for i in range(32 * 4)]
for xo in range(32):
    for xi in range(4):
        C[xo * 4 + xi] = A[xo * 4 + xi] + B[max(xo * 4 + xi - 1, 0)]
        C_idx[xo * 4 + xi]= [xo * 4 + xi, max(xo * 4 + xi - 1, 0)]

In [ ]:
C2 = [0 for i in range(32 * 4)]
C2_idx = [[] for i in range(32 * 4)]
for xi in range(4):
    for xo in range(32):
        C2[xo * 4 + xi] = A[xo * 4 + xi] + B[max(xo * 4 + xi - 1, 0)]
        C2_idx[xo * 4 + xi] = [xo * 4 + xi, max(xo * 4 + xi - 1, 0)]
   

In [ ]:
def diff_val(l1, l2):
    diff_list = []
    for idx, item_l1 in enumerate(l1):
        item_l2 = l2[idx]
        diff_list.append(item_l2 - item_l1)
    return diff_list

In [ ]:
def diff_idx(l1, l2):
    diff_list = []
    for idx, item_l1 in enumerate(l1):
        item_l2 = l2[idx]
        diff_list.append([item_l2[0] - item_l1[0], item_l2[1] - item_l1[1]])
    return diff_list

In [ ]:
# res = sum(diff(C, C2))
# res = diff_idx(C_idx, C2_idx)
# both are zero, wtf?

### Toy sample TVM

In [ ]:
!python3 -m  pip install mlc-ai-nightly -f https://mlc.ai/wheels


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://mlc.ai/wheels


In [ ]:
import numpy as np
import tvm
from tvm.ir.module import IRModule
from tvm.script import tir as T

In [ ]:
@tvm.script.ir_module
class MyModule:
    @T.prim_func
    def main(A: T.Buffer[128, "float32"],
             B: T.Buffer[128, "float32"],
             C: T.Buffer[128, "float32"]):
        # extra annotations for the function
        T.func_attr({"global_symbol": "main", "tir.noalias": True})
        for i in range(128):
            with T.block("C"):
                # declare a data parallel iterator on spatial domain
                vi = T.axis.spatial(128, i)
                C[vi] = A[vi] + B[vi]

In [ ]:
sch = tvm.tir.Schedule(MyModule)

In [ ]:
block_c = sch.get_block("C")

In [ ]:
i, = sch.get_loops(block_c)

In [ ]:
from tvm import te

M = 1024
K = 1024
N = 1024

# The default tensor type in tvm
dtype = "float32"

target = "llvm"
dev = tvm.device(target, 0)

# Algorithm
k = te.reduce_axis((0, K), "k")
A = te.placeholder((M, K), name="A")
B = te.placeholder((K, N), name="B")
C = te.compute((M, N), lambda m, n: te.sum(A[m, k] * B[k, n], axis=k), name="C")

# Default schedule
func = te.create_prim_func([A, B, C])
func = func.with_attr("global_symbol", "main")
ir_module = IRModule({"main": func})
print(ir_module.script())


func = tvm.build(ir_module, target="llvm")  # The module for CPU backends.

a = tvm.nd.array(np.random.rand(M, K).astype(dtype), dev)
b = tvm.nd.array(np.random.rand(K, N).astype(dtype), dev)
c = tvm.nd.array(np.zeros((M, N), dtype=dtype), dev)
func(a, b, c)

evaluator = func.time_evaluator(func.entry_name, dev, number=1)
print("Baseline: %f" % evaluator(a, b, c).mean)

@tvm.script.ir_module
class Module:
    @tir.prim_func
    def func(A: tir.Buffer[(1024, 1024), "float32"], B: tir.Buffer[(1024, 1024), "float32"], C: tir.Buffer[(1024, 1024), "float32"]) -> None:
        # function attr dict
        tir.func_attr({"global_symbol": "main", "tir.noalias": True})
        # body
        # with tir.block("root")
        for i0, i1, i2 in tir.grid(1024, 1024, 1024):
            with tir.block("C"):
                m, n, k = tir.axis.remap("SSR", [i0, i1, i2])
                tir.reads(A[m, k], B[k, n])
                tir.writes(C[m, n])
                with tir.init():
                    C[m, n] = tir.float32(0)
                C[m, n] = C[m, n] + A[m, k] * B[k, n]
    
Baseline: 4.361387


In [ ]:
sch = tvm.tir.Schedule(ir_module)
type(sch)
block_c = sch.get_block("C")
# Get loops surrounding the block
(y, x, k) = sch.get_loops(block_c)
block_size = 32
yo, yi = sch.split(y, [None, block_size])
xo, xi = sch.split(x, [None, block_size])

sch.reorder(yo, xo, k, yi, xi)
print(sch.mod.script())

func = tvm.build(sch.mod, target="llvm")  # The module for CPU backends.

c = tvm.nd.array(np.zeros((M, N), dtype=dtype), dev)
func(a, b, c)

evaluator = func.time_evaluator(func.entry_name, dev, number=1)
print("after transformation: %f" % evaluator(a, b, c).mean)

@tvm.script.ir_module
class Module:
    @tir.prim_func
    def func(A: tir.Buffer[(1024, 1024), "float32"], B: tir.Buffer[(1024, 1024), "float32"], C: tir.Buffer[(1024, 1024), "float32"]) -> None:
        # function attr dict
        tir.func_attr({"global_symbol": "main", "tir.noalias": True})
        # body
        # with tir.block("root")
        for i0_0, i1_0, i2, i0_1, i1_1 in tir.grid(32, 32, 1024, 32, 32):
            with tir.block("C"):
                m = tir.axis.spatial(1024, i0_0 * 32 + i0_1)
                n = tir.axis.spatial(1024, i1_0 * 32 + i1_1)
                k = tir.axis.reduce(1024, i2)
                tir.reads(A[m, k], B[k, n])
                tir.writes(C[m, n])
                with tir.init():
                    C[m, n] = tir.float32(0)
                C[m, n] = C[m, n] + A[m, k] * B[k, n]
    
after transformation: 0.390557


In [ ]:
sch.parallel(yo)
print(sch.mod.script())
func = tvm.build(sch.mod, target="llvm")

@tvm.script.ir_module
class Module:
    @tir.prim_func
    def func(A: tir.Buffer[(1024, 1024), "float32"], B: tir.Buffer[(1024, 1024), "float32"], C: tir.Buffer[(1024, 1024), "float32"]) -> None:
        # function attr dict
        tir.func_attr({"global_symbol": "main", "tir.noalias": True})
        # body
        # with tir.block("root")
        for i0_0 in tir.parallel(32):
            for i1_0, i2, i0_1, i1_1 in tir.grid(32, 1024, 32, 32):
                with tir.block("C"):
                    m = tir.axis.spatial(1024, i0_0 * 32 + i0_1)
                    n = tir.axis.spatial(1024, i1_0 * 32 + i1_1)
                    k = tir.axis.reduce(1024, i2)
                    tir.reads(A[m, k], B[k, n])
                    tir.writes(C[m, n])
                    with tir.init():
                        C[m, n] = tir.float32(0)
                    C[m, n] = C[m, n] + A[m, k] * B[k, n]
    
